In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
uploaded = files.upload()

Saving user-topic-qa-data(new).xlsx to user-topic-qa-data(new).xlsx


In [ ]:
############## Reading data ######################

raw_data = pd.read_excel('user-topic-qa-data (1).xlsx')
data = raw_data.copy()


In [ ]:
# since some data(some features of a row) is/are misplaced into next row

for i in range(len(data)):
  if type(data['question'][i])!=int:
    data['options'][i-1]=data['question'][i]
    data['correct_responses'][i-1]=data['correct'][i]
    data['difficulty_level'][i-1]=data['user'][i]
    data['exam'][i-1]=data['time_spend'][i]
    data['level'][i-1]=data['answers'][i]
    data['group_concat(distinct (qc.concept))'][i-1]=data['text'][i]
    data['group_concat(distinct (ecm.topic))'][i-1]=data['options'][i]
    data.drop(labels=i,axis=0,inplace=True)
data.reset_index(drop=True,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

# .

In [ ]:
data['correct'].fillna(0, inplace=True)
data.head()

,modified_on,question,correct,user,time_spend,answers,text,options,correct_responses,difficulty_level,exam,level,group_concat(distinct (qc.concept)),group_concat(distinct (ecm.topic))
0,2020-10-22 05:33:41.100,1763,0,28.0,58,[],"<p>Find the average of 1.1, 0.01, 0.101, 0.001...","[""<p>2.664</p>"",""<p>1.2664</p>"",""<p>0.2664</p>...","[""2""]",1.0,RRB_NTPC,1.55,AVERAGES,AVERAGES
1,2020-10-22 05:35:43.241,1765,0,28.0,156,"[""2""]","<p>The average of 1, 3, 5, 7, 9, 11 ……upto 25 ...","[""<p>125</p>"",""<p>15</p>"",""<p>625</p>"",""<p>25<...","[""3""]",1.5,RRB_NTPC,1.55,AVERAGES,AVERAGES
2,2020-10-22 05:35:50.140,1769,1,28.0,51,"[""0""]",<p>The average of cube of the first 10 natural...,"[""<p>100</p>"",""<p>304.5</p>"",""<p>302.5</p>"",""<...","[""1""]",2.5,RRB_NTPC,1.55,AVERAGES,AVERAGES
3,2020-10-22 05:34:34.614,1760,0,28.0,95,"[""1""]",<p>The average marks obtained by 8 students we...,"[""<p>58</p>"",""<p>59</p>"",""<p>&nbsp;60</p>"",""<p...","[""2""]",1.5,RRB_NTPC,1.55,AVERAGES,AVERAGES
4,2020-10-22 05:31:23.420,1764,0,28.0,8,"[""1""]","<p>Out of 20 boys, 6 boys are each of 1m 15 cm...","[""<p>1m 12.1cm</p>"",""<p>1m 21.1cm</p>"",""<p>1m ...","[""0""]",1.5,RRB_NTPC,1.55,AVERAGES,AVERAGES


In [ ]:
# changing type from object to datetime
# encoding topic and exam name to numerics
# changing skip (NULL) to 0, correct to 1 and wrong to -1
# all these changes will reduce memory usage and make data easy to handle
# there are many exact repetitions in values removing because they will induce bias in things like average time

exam_names = ['RRB_NTPC', 'SSC_CHSL', 'SSC_CGL']    # list of exam names
data = data.rename(columns = {"group_concat(distinct (ecm.topic))":"topic"})
topic_names = data.topic.unique()                  # list of topic names

# encoding names and also changing wrong from 0 to -1 then skip from null to 0 and dropping exact duplicates
data2 = (data.replace(to_replace = {'topic' : topic_names, 'exam' : exam_names, 'correct' : 0}, 
                    value = {'topic' : range(len(topic_names)), 'exam' : range(len(exam_names)), 'correct' : -1})
        .dropna()
        .drop_duplicates())
data2['options'][data2.options == '[]']='nil'
print(len(data['question'].unique()))



5727


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [ ]:
data2['correct_option_text']=' '
data2['wrong_options_text']=' '
def do_preprocess(df1):
    for index, row in df1.iterrows():
        filter_sentence = ''
        question = row['text']
        options=row['options']
        correctresponse=row['correct_responses']
 
        sentence = ' ' + question
 
 
        # Cleaning the sentence with regex
        sentence = sentence.replace('<p>','')#re.sub('<p>', ' ', sentence)
        sentence = sentence.replace('</p>','')#re.sub('</p>', '', sentence)
        sentence=sentence.replace("</strong>",'')
        sentence=sentence.replace("br",'')
        cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
        sentence = re.sub(cleanr, '', sentence)
        sentence = re.sub('[^a-zA-Z0-9.]', ' ', sentence)
        
        options = re.sub(r'<p>', ' ', options)
        options = re.sub(r'</p>', '#', options)
        options=options.split('#')
        # options = re.sub('[^a-zA-Z0-9.,]', ' ', options)
 
 
        correctresponse = re.sub('[^a-zA-Z0-9.]', ' ', correctresponse)
 
        # Tokenization
        words = nltk.word_tokenize(sentence)
 
        # Stopwords removal
        stop_words=set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]
        
        for words in words:
            filter_sentence = filter_sentence  + ' ' + str(lemmatizer.lemmatize(words)).lower()
        print(index)
        # print(sentence)
        # print(options)
        w=''
        for i in range(len(options)):
          if i!=int(correctresponse):
            options[i] = re.sub('[^a-zA-Z0-9.,]', ' ', options[i])
            options[i]=options[i].replace("br",'')
            # to remove nbsp in correct response ext
            # cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
            # options[i] = re.sub(cleanr, '',options[i])
            w=w+options[i]
        # print(options)
        # ---------------------------------------------
        temp= re.sub("[^a-zA-Z]","",str(options))
        if temp=='nil':
          correctresponse=''
        else:
          correctresponse = re.sub('[^a-zA-Z0-9.]', ' ', options[int(correctresponse)])
          correctresponse=correctresponse.replace('nbsp','')
          # -----------------------------------------------------
        # print(w)
        df1.loc[index, 'text'] = filter_sentence
        df1.loc[index, 'correct_option_text'] = correctresponse
        df1.loc[index, 'wrong_options_text'] =w
    df1 = df1[['text','correct_option_text','wrong_options_text','topic']]
    return(df1)
df=do_preprocess(data2)

In [ ]:
df.head()

,text,correct_option_text,wrong_options_text,topic
0,Find the average of 1.1 0.01 0.101 0.001 ...,0.2664,"2.664 , 1.2664 , 0.2554",0
1,The average of 1 3 5 7 9 11 upto 25 t...,25,"125 , 15 , 625",0
2,The average of cube of the first 10 natural ...,304.5,"100 , 302.5 , 380",0
3,The average marks obtained by 8 students wer...,60,"58 , 59 , 61",0
4,Out of 20 boys 6 boys are each of 1m 15 cm ...,1m 12.1cm,", 1m 21.1cm , 1m 21cm nbsp , 1m 12cm",0
...,...,...,...,...
2080,Which of the following persons are sitting ...,GFD,"DAB , BCA , GCE",6
2081,Which of the following is correct,B is between A and D,", D is between F and G , nbsp E is to the ...",6
2082,If given 12 169 5 4 20 2 and 1 26 ...,58,"78 , 68 , 36",16
2083,Which of the following symbols should replac...,,nil,16


In [ ]:
# data2.head()
finaldata=data2[['question','text','difficulty_level','options','correct_option_text','wrong_options_text']]
finaldata.head()

,question,text,difficulty_level,options,correct_option_text,wrong_options_text
0,1763,Find the average of 1.1 0.01 0.101 0.001 ...,1.0,"[""<p>2.664</p>"",""<p>1.2664</p>"",""<p>0.2664</p>...",0.2664,"2.664 , 1.2664 , 0.2554"
1,1765,The average of 1 3 5 7 9 11 upto 25 t...,1.5,"[""<p>125</p>"",""<p>15</p>"",""<p>625</p>"",""<p>25<...",25,"125 , 15 , 625"
2,1769,The average of cube of the first 10 natural ...,2.5,"[""<p>100</p>"",""<p>304.5</p>"",""<p>302.5</p>"",""<...",304.5,"100 , 302.5 , 380"
3,1760,The average marks obtained by 8 students wer...,1.5,"[""<p>58</p>"",""<p>59</p>"",""<p>&nbsp;60</p>"",""<p...",60,"58 , 59 , 61"
4,1764,Out of 20 boys 6 boys are each of 1m 15 cm ...,1.5,"[""<p>1m 12.1cm</p>"",""<p>1m 21.1cm</p>"",""<p>1m ...",1m 12.1cm,", 1m 21.1cm , 1m 21cm nbsp , 1m 12cm"


In [ ]:
finaldata.to_csv('afterpreprocess_without_stopwords.csv')

# pre processing the dataset the for girth implementation, should be in format items*reponses at the end

In [ ]:
finaldata4=data2[['user','question','correct']].sort_values('question')
# print(finaldata4.groupby('question')['correct'].count()>50)
questionind=finaldata4['question'].unique()
s=[]     #array of responses of qns having 50+ responses 
ind=[]   #Question ids of these particular qns
response_rate=[]
for i in questionind:
  arr=[]
  item=finaldata4[['correct']][finaldata4['question']==i]
  k=0
  a=0
  if len(item)>50:
    ind.append(i)
    for j in item["correct"]:
      if k<=50:
        if j==1:
          arr.append(True)
          a=a+1
          k=k+1
        if j==-1:
          arr.append(False)
          k=k+1
    response_rate.append(a/50)
    s=s+[arr]

In [ ]:
ss=np.stack(s, axis=0)
print(len(ss))

1


In [ ]:
pip install girth

  Created wheel for girth: filename=girth-0.3.7-cp37-none-any.whl size=42737 sha256=0d53db6af8101063880a012f82c5c103f9a78fcc2bd0ecfb56e42861780a24f4
  Stored in directory: /root/.cache/pip/wheels/da/40/9f/5bc8547c121668f72423ab17930131b47247cbf77bc9c079cc
Successfully built girth


In [ ]:

import numpy as np
from scipy.optimize import fminbound, fmin_slsqp
from scipy import integrate

from girth import (validate_estimation_options, get_true_false_counts, 
                   convert_responses_to_kernel_sign)
from girth.mml_methods import _mml_abstract
from girth.utils import _get_quadrature_points
from girth.three_pl.three_pl_utils import _compute_partial_integral_3pl


def threepl_mml(dataset, options=None):
    """ Estimates parameters in a 3PL IRT model.
    Args:
        dataset: [items x participants] matrix of True/False Values
        options: dictionary with updates to default options
    Returns:
        discrimination: (1d array) estimate of item discriminations
        difficulty: (1d array) estimates of item diffiulties
        guessing: (1d array) estimates of item guessing
    
    Options:
        * max_iteration: int
        * distribution: callable
        * quadrature_bounds: (float, float)
        * quadrature_n: int
    """
    options = validate_estimation_options(options)
    quad_start, quad_stop = options['quadrature_bounds']
    quad_n = options['quadrature_n']

    n_items = dataset.shape[0]
    n_no, n_yes = get_true_false_counts(dataset)
    scalar = n_yes / (n_yes + n_no)
    
    unique_sets, counts = np.unique(dataset, axis=1, return_counts=True)
    the_sign = convert_responses_to_kernel_sign(unique_sets)

    theta, weights = _get_quadrature_points(quad_n, quad_start, quad_stop)
    distribution = options['distribution'](theta)
    distribution_x_weights = distribution * weights

    # Perform the minimization
    discrimination = np.ones((n_items,))
    difficulty = np.zeros((n_items,))
    guessing = np.zeros((n_items,))
    
    local_scalar = np.zeros((1, 1))

    for iteration in range(options['max_iteration']):
        previous_discrimination = discrimination.copy()

        # Quadrature evaluation for values that do not change
        # This is done during the outer loop to address rounding errors
        partial_int = _compute_partial_integral_3pl(theta, difficulty,
                                                discrimination, guessing, the_sign)
        partial_int *= distribution

        for ndx in range(n_items):
            # pylint: disable=cell-var-from-loop

            # remove contribution from current item
            local_int = _compute_partial_integral_3pl(theta, difficulty[ndx, None],
                                                  discrimination[ndx, None], 
                                                  guessing[ndx, None],
                                                  the_sign[ndx, None])

            partial_int /= local_int

            def min_func_local(estimate):
                discrimination[ndx] = estimate[0]
                guessing[ndx] = estimate[1]
                
                local_scalar[0, 0] = (scalar[ndx] - guessing[ndx]) / (1. - guessing[ndx])
                _mml_abstract(difficulty[ndx, None], local_scalar,
                              discrimination[ndx, None], theta, distribution_x_weights)
                estimate_int = _compute_partial_integral_3pl(theta, difficulty[ndx, None],
                                                         discrimination[ndx, None],
                                                         guessing[ndx, None],
                                                         the_sign[ndx, None])

                estimate_int *= partial_int
                otpt = integrate.fixed_quad(
                    lambda x: estimate_int, quad_start, quad_stop, n=quad_n)[0]
                
                return -np.log(otpt).dot(counts)

            # Solve for the discrimination parameters
            initial_guess = [discrimination[ndx], guessing[ndx]]
            fmin_slsqp(min_func_local, initial_guess, 
                       bounds=([0.25, 4], [0, .33]), iprint=False)

            # Update the partial integral based on the new found values
            estimate_int = _compute_partial_integral_3pl(theta, difficulty[ndx, None],
                                                     discrimination[ndx, None],
                                                     guessing[ndx, None], 
                                                     the_sign[ndx, None])
            # update partial integral
            partial_int *= estimate_int

        if np.abs(discrimination - previous_discrimination).max() < 1e-3:
            break

    return {'Discrimination': discrimination, 
            'Difficulty': difficulty, 
            'Guessing': guessing}

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
est_params=threepl_mml(ss, options=None)

In [ ]:
from girth import twopl_mml
est_params_twopl=twopl_mml(ss,options=None)
from girth import onepl_mml
est_params_onepl=onepl_mml(ss,options=None)

In [ ]:
#est_params

In [ ]:
#est_params_twopl

In [ ]:
#est_params_onepl

In [ ]:
finalData=finaldata[finaldata['question'].isin(ind)]
finalData.shape

In [ ]:
FinalData=finalData.drop_duplicates('question')
FinalData.shape

In [ ]:
testdata=finaldata[finaldata['question'].isin(ind)==False]
testdata.shape

In [ ]:
test_data=testdata.drop_duplicates('question')
test_data.shape

In [ ]:
est_params=est_params_twopl

In [ ]:
FinalData=FinalData.sort_values('question')
FinalData['complete_text']=data['text'][data['question'].isin(ind)]
# df = pddf['column_name'].
FinalData['diificulty_girth']=pd.DataFrame(est_params['Difficulty']).values
FinalData['descrimination_girth']=pd.DataFrame(est_params['Discrimination']).values
FinalData['guessing_girth']=pd.DataFrame(est_params['Guessing']).values

print(len(FinalData['question'].unique()))
# print(len(df['column_name']))
FinalData.head()
# df.head()


### NLP Model

In [ ]:
from sklearn.model_selection import train_test_split
train_text,val_text,train_labels,val_labels=train_test_split(FinalData['text'],FinalData[['diificulty_girth','descrimination_girth','guessing_girth'
]],test_size=0.1)
print(train_text.shape)
print(val_text.shape)
print(train_labels.shape)
print(val_labels.shape)

(417,)
(47,)
(417, 3)
(47, 3)


In [ ]:
test_text=test_data['text']
test_text.shape

(5263,)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(ngram_range=(1,1),use_idf=True)
mat=tfidf.fit_transform(train_text).toarray()
train_df=pd.DataFrame(mat,columns=tfidf.get_feature_names())
val_df=pd.DataFrame(tfidf.transform(val_text).toarray(),columns=tfidf.get_feature_names())
test_df=pd.DataFrame(tfidf.transform(test_text).toarray(),columns=tfidf.get_feature_names())
train_df.head()

,032,0o,10,100,1000,10000,10km,11,12,12000,121212,13,130,130768,133,13km,14,147,15,15000,157,16,17,17500,18,180,182,1860,19,190,192,1930,20,2000,2003,2012,2014,2016,2019,2020,...,who,wife,wild,will,willpower,win,window,winds,wire,with,withdrew,within,without,woman,women,won,wood,word,words,work,world,worship,worth,would,writer,written,wrote,www,yahoo,year,years,yeknod,you,young,your,youthful,yz,zap,zeros,zip
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.264332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.555219,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.2699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.ensemble import RandomForestRegressor
model11=RandomForestRegressor(max_features='sqrt')
model21=RandomForestRegressor(max_features='sqrt')
model31=RandomForestRegressor(max_features='sqrt')
model11.fit(train_df,train_labels['descrimination_girth'])
model21.fit(train_df,train_labels['diificulty_girth'])
model31.fit(train_df,train_labels['guessing_girth'])

from sklearn.metrics import mean_squared_error
print(mean_squared_error(val_labels['descrimination_girth'],model11.predict(val_df)))
print(mean_squared_error(val_labels['diificulty_girth'],model21.predict(val_df)))
print(mean_squared_error(val_labels['guessing_girth'],model31.predict(val_df)))

1.168653495708271
8.025486819501786
0.02770537388921892


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler((1,5))
train_labels[['diificulty_girth']]=scaler.fit_transform(train_labels[['diificulty_girth']])
val_labels[['diificulty_girth']]=scaler.transform(val_labels[['diificulty_girth']])

scaler2=MinMaxScaler((0,1))
train_labels[['descrimination_girth']]=scaler.fit_transform(train_labels[['descrimination_girth']])
val_labels[['descrimination_girth']]=scaler.transform(val_labels[['descrimination_girth']])

scaler3=MinMaxScaler((0.25,0.4))
train_labels[['guessing_girth']]=scaler.fit_transform(train_labels[['guessing_girth']])
val_labels[['guessing_girth']]=scaler.transform(val_labels[['guessing_girth']])